<a href="https://colab.research.google.com/github/KarandikarA/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Preprocessing

In [104]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [61]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [62]:
# Determine the number of unique values in each column.
application_df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [63]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [64]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [65]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [66]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 1]


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [67]:
# Choose a cutoff value and create a list of classifications to be replaced
values_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = values_counts[values_counts <= 777].index.tolist()

# Replace in dataframe
for classification in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classification,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [68]:
# Replace the SPECIAL_CONSIDERATIONS values with 0 and 1
application_df['SPECIAL_CONSIDERATIONS'] = application_df['SPECIAL_CONSIDERATIONS'].replace({'N':0, 'Y':1})

In [69]:
# Convert categorical data to numeric with `pd.get_dummies`
encoded_df = pd.get_dummies(application_df[["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT"]])
final_df = application_df.merge(encoded_df, left_index=True, right_index=True)
final_df = final_df.drop(columns=["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT"])
final_df.head()

,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,0,5000,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,108590,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,5000,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,1,0,6692,1,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,0,142590,1,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0


In [70]:
# Split our preprocessed data into our features and target arrays
y = final_df["IS_SUCCESSFUL"]
X = final_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [71]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

### Optimzing the model

In [72]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model_1 = tf.keras.models.Sequential()

# First hidden layer
nn_model_1.add(tf.keras.layers.Dense(units=120, activation="relu", input_dim=42))

# Second hidden layer
nn_model_1.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Output layer
nn_model_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_1.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 120)               5160      
                                                                 
 dense_24 (Dense)            (None, 80)                9680      
                                                                 
 dense_25 (Dense)            (None, 1)                 81        
                                                                 
Total params: 14921 (58.29 KB)
Trainable params: 14921 (58.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [73]:
# Compile the model
nn_model_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [74]:
# Train the model
fit_model = nn_model_1.fit(X_train_scaled, y_train, epochs=2)

Epoch 1/2
804/804 [==============================] - 4s 3ms/step - loss: 0.5714 - accuracy: 0.7238
Epoch 2/2
804/804 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7296


In [75]:
# Evaluate the model using the test data
model_loss_1, model_accuracy_1 = nn_model_1.evaluate(X_test_scaled,y_test,verbose=2, batch_size=1 )
print(f"Loss: {model_loss_1}, Accuracy: {model_accuracy_1}")

8575/8575 - 12s - loss: 0.5613 - accuracy: 0.7221 - 12s/epoch - 1ms/step
Loss: 0.5613127946853638, Accuracy: 0.7220991253852844


In [76]:
# Export our model to HDF5 file
nn_model_1.save("Outputs/Optimizations/AlphabetSoupCharity_Xtra_neurons.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Running Model with more layers

In [77]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=42))
# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu"))
# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 80)                3440      
                                                                 
 dense_27 (Dense)            (None, 80)                6480      
                                                                 
 dense_28 (Dense)            (None, 30)                2430      
                                                                 
 dense_29 (Dense)            (None, 1)                 31        
                                                                 
Total params: 12381 (48.36 KB)
Trainable params: 12381 (48.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [78]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [79]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=2)


Epoch 1/2
804/804 [==============================] - 3s 2ms/step - loss: 0.5697 - accuracy: 0.7226
Epoch 2/2
804/804 [==============================] - 2s 2ms/step - loss: 0.5557 - accuracy: 0.7310


In [80]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 1s - loss: 0.5610 - accuracy: 0.7262 - 511ms/epoch - 2ms/step
Loss: 0.5609826445579529, Accuracy: 0.7261807322502136


In [81]:
# Export our model to HDF5 file
nn_model.save("Outputs/Optimizations/AlphabetSoupCharity_Xtra_layer.h5")


### Optimizing Model with different activation function

In [82]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model_2 = tf.keras.models.Sequential()

# First hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=80, activation="tanh", input_dim=42))
# Second hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=80, activation="tanh"))

# Output layer
nn_model_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_2.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 80)                3440      
                                                                 
 dense_31 (Dense)            (None, 80)                6480      
                                                                 
 dense_32 (Dense)            (None, 1)                 81        
                                                                 
Total params: 10001 (39.07 KB)
Trainable params: 10001 (39.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [83]:
# Compile the model
nn_model_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [84]:
# Train the model
fit_model = nn_model_2.fit(X_train_scaled, y_train, epochs=2)

Epoch 1/2
804/804 [==============================] - 2s 2ms/step - loss: 0.5747 - accuracy: 0.7191
Epoch 2/2
804/804 [==============================] - 2s 2ms/step - loss: 0.5590 - accuracy: 0.7283


In [85]:
# Evaluate the model using the test data
model_2_loss, model_2_accuracy = nn_model_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_2_loss}, Accuracy: {model_2_accuracy}")

268/268 - 1s - loss: 0.5634 - accuracy: 0.7250 - 681ms/epoch - 3ms/step
Loss: 0.5634351372718811, Accuracy: 0.7250145673751831


In [86]:
# Export our model to HDF5 file
nn_model_2.save("Outputs/Optimizations/AlphabetSoupCharity_tanh.h5")

### Optimzing model with more epochs

In [87]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model_3 = tf.keras.models.Sequential()

# First hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=42))

# Second hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_model_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_3.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 80)                3440      
                                                                 
 dense_34 (Dense)            (None, 30)                2430      
                                                                 
 dense_35 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5901 (23.05 KB)
Trainable params: 5901 (23.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [88]:
# Compile the model
nn_model_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [89]:
# Train the model
fit_model = nn_model_3.fit(X_train_scaled, y_train, epochs=4)

Epoch 1/4
804/804 [==============================] - 3s 2ms/step - loss: 0.5690 - accuracy: 0.7211
Epoch 2/4
804/804 [==============================] - 2s 2ms/step - loss: 0.5554 - accuracy: 0.7280
Epoch 3/4
804/804 [==============================] - 2s 2ms/step - loss: 0.5532 - accuracy: 0.7321
Epoch 4/4
804/804 [==============================] - 2s 2ms/step - loss: 0.5513 - accuracy: 0.7309


In [90]:
# Evaluate the model using the test data
model_3_loss, model_3_accuracy = nn_model_3.evaluate(X_test_scaled,y_test,verbose=2, batch_size=1 )
print(f"Loss: {model_3_loss}, Accuracy: {model_3_accuracy}")

8575/8575 - 12s - loss: 0.5566 - accuracy: 0.7265 - 12s/epoch - 1ms/step
Loss: 0.5566271543502808, Accuracy: 0.7265306115150452


In [91]:
# Export our model to HDF5 file
nn_model_3.save("Outputs/Optimizations/AlphabetSoupCharity_moreEpoch.h5")

### Optimizing Model by dropping features

In [92]:
# Import Dependencies
# Using the RandomForestClassifier to create random forest classifier
# Identifies the least important features in model
from sklearn.ensemble import RandomForestClassifier

model_RF = RandomForestClassifier()
model_RF.fit(X_train_scaled, y_train)
importances = model_RF.feature_importances_

feature_importances = zip(X_train.columns, importances)
sorted_features = sorted(feature_importances, key=lambda x: x[1], reverse=True)

for feature, importance in sorted_features:
    print(f"Features: {feature}, Importance: {importance}")

Features: ASK_AMT, Importance: 0.4032971843139833
Features: AFFILIATION_Independent, Importance: 0.13994960980411983
Features: AFFILIATION_CompanySponsored, Importance: 0.11706206055003607
Features: ORGANIZATION_Association, Importance: 0.030669643092620557
Features: APPLICATION_TYPE_T5, Importance: 0.025049653633265168
Features: APPLICATION_TYPE_T10, Importance: 0.02383800238300625
Features: CLASSIFICATION_Other, Importance: 0.020541015928887196
Features: CLASSIFICATION_C2100, Importance: 0.018659979723563605
Features: ORGANIZATION_Trust, Importance: 0.018445969889020628
Features: APPLICATION_TYPE_T19, Importance: 0.01812797021294642
Features: CLASSIFICATION_C2000, Importance: 0.01736252885024654
Features: APPLICATION_TYPE_T4, Importance: 0.016023926036629447
Features: APPLICATION_TYPE_T3, Importance: 0.015976218807388202
Features: CLASSIFICATION_C1000, Importance: 0.01335040489610996
Features: APPLICATION_TYPE_T6, Importance: 0.012177370133591386
Features: USE_CASE_Preservation, Impo

In [93]:
# Dropping Noise columns
low_importance_features = ['SPECIAL_CONSIDERATIONS', 'STATUS']

# Dropping low importance features in X
X_train_reduced = X_train.drop(low_importance_features, axis=1)
X_test_reduced = X_test.drop(low_importance_features, axis=1)


In [94]:
# Creating the StandardScaler instances
scaler = StandardScaler()

# Fiting the StandardScaler
X_scaler_reduced = scaler.fit(X_train_reduced)

# Scaling the data
X_train_scaled_reduced = X_scaler_reduced.transform(X_train_reduced)
X_test_scaled_reduced = X_scaler_reduced.transform(X_test_reduced)

In [95]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model_4 = tf.keras.models.Sequential()

# First hidden layer
nn_model_4.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=40))

# Second hidden layer
nn_model_4.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_model_4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_4.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 80)                3280      
                                                                 
 dense_37 (Dense)            (None, 30)                2430      
                                                                 
 dense_38 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5741 (22.43 KB)
Trainable params: 5741 (22.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [96]:
# Compile the model
nn_model_4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [97]:
# Train the model
fit_model = nn_model_4.fit(X_train_scaled_reduced, y_train, epochs=2)

Epoch 1/2
804/804 [==============================] - 4s 3ms/step - loss: 0.5710 - accuracy: 0.7222
Epoch 2/2
804/804 [==============================] - 2s 3ms/step - loss: 0.5553 - accuracy: 0.7303


In [98]:
# Evaluate the model using the test data
model_4_loss, model_4_accuracy = nn_model_4.evaluate(X_test_scaled_reduced,y_test,verbose=2, batch_size=1 )
print(f"Loss: {model_4_loss}, Accuracy: {model_4_accuracy}")

8575/8575 - 14s - loss: 0.5599 - accuracy: 0.7247 - 14s/epoch - 2ms/step
Loss: 0.5599087476730347, Accuracy: 0.7246647477149963


In [99]:
# Export our model to HDF5 file
nn_model_4.save("Outputs/Optimizations/AlphabetSoupCharity_Drop_Noise.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Optimizing model with Keras Tuner

In [100]:
# Creating the method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model_5 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model_5.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=120,
        step=10), activation=activation, input_dim=42))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model_5.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=120,
            step=10),
            activation=activation))

    nn_model_5.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model_5.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model_5

In [106]:
# Import the kerastuner library
!pip install -q -U keras-tuner
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=10,
    hyperband_iterations=2)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.3 MB/s eta 0:00:00


In [107]:
# Running kerastuner search to get best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=10,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 30s]
val_accuracy: 0.7255976796150208

Best val_accuracy So Far: 0.7290962338447571
Total elapsed time: 00h 16m 08s


In [109]:
# Finding the best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 111,
 'num_layers': 1,
 'units_0': 11,
 'units_1': 31,
 'units_2': 111,
 'units_3': 71,
 'units_4': 71,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [110]:
# Evaluating the best model with the full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5535 - accuracy: 0.7291 - 574ms/epoch - 2ms/step
Loss: 0.5535464286804199, Accuracy: 0.7290962338447571


In [111]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model_tuner = tf.keras.models.Sequential()

# First hidden layer
nn_model_tuner.add(tf.keras.layers.Dense(units=81, activation="relu", input_dim=42))

# Second hidden layer
nn_model_tuner.add(tf.keras.layers.Dense(81, activation='relu'))

# Third hidden layer
nn_model_tuner.add(tf.keras.layers.Dense(21, activation='relu'))

# Output layer
nn_model_tuner.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_tuner.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 81)                3483      
                                                                 
 dense_4 (Dense)             (None, 81)                6642      
                                                                 
 dense_5 (Dense)             (None, 21)                1722      
                                                                 
 dense_6 (Dense)             (None, 1)                 22        
                                                                 
Total params: 11869 (46.36 KB)
Trainable params: 11869 (46.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [112]:
# Compile the model
nn_model_tuner.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [113]:
# Train the model
fit_model = nn_model_tuner.fit(X_train_scaled, y_train, epochs=2)

Epoch 1/2
804/804 [==============================] - 3s 2ms/step - loss: 0.5700 - accuracy: 0.7199
Epoch 2/2
804/804 [==============================] - 2s 2ms/step - loss: 0.5555 - accuracy: 0.7297


In [114]:
# Evaluate the model using the test data
model_tuner_loss, model_tuner_accuracy = nn_model_tuner.evaluate(X_test_scaled,y_test,verbose=2, batch_size=1 )
print(f"Loss: {model_tuner_loss}, Accuracy: {model_tuner_accuracy}")

8575/8575 - 17s - loss: 0.5552 - accuracy: 0.7276 - 17s/epoch - 2ms/step
Loss: 0.5551783442497253, Accuracy: 0.727580189704895


In [115]:
# Export our model to HDF5 file
nn_model_tuner.save("Outputs/Optimizations/AlphabetSoupCharity_keras_tuner.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
